# Librerías

In [3]:
from collections import defaultdict
import pandas as pd
import ipywidgets as widgets
from sklearn.metrics import pairwise_distances
import heapq
import numpy as np
import random as rd

# Importar datos preprocesados

In [4]:
track_features = pd.read_csv('./data/processed_track_features.csv')
sessions = pd.read_csv('./data/processed_sessions.csv')
display(track_features.head(3))
display(sessions.head(3))

,Unnamed: 0,track_id,duration,year,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0,t_a540e552-16d4-42f8-a185-232bd650ea7d,0.061365,0.964392,0.458040,0.399767,0.817709,3.254327e-06,0,0.132124,0.187300,0,0.079985,0.760082,4,0.935512
1,1,t_67965da0-132b-4b1e-8a69-0ef99b32287c,0.104988,0.964392,0.916272,0.491235,0.154258,8.344854e-12,0,0.163281,0.228433,0,0.083877,0.435429,3,0.359675
2,2,t_0614ecd3-a7d5-40a1-816e-156d5872a467,0.089967,0.964886,0.812884,0.491625,0.358813,2.927475e-10,0,0.090115,0.175367,1,0.038777,0.480795,4,0.726769


,user_id,track_id,not_skipped
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,True
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,True
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,True


In [5]:
track_features_embedding = track_features.drop(labels="track_id", axis=1)

In [6]:
idx_to_track_id = {i: id_ for i, id_ in enumerate(track_features.track_id)}
track_id_to_idx = {id_:i for i, id_ in enumerate(track_features.track_id)}

In [8]:
idx_to_sessions_id = {i: id_ for i, id_ in enumerate(sessions.user_id)}
sessions_id_to_idx = {id_:i for i, id_ in enumerate(sessions.user_id)}

# Nuestro modelo

# Crear Top N verdadero por usuario

In [ ]:
# top_n_verdadero_por_usuario es un diccionario de la forma
# {usuario1: [canción1, canción2, ...], usuario2: [canción3, canción4,...], ...}
# que contiene las canciones con las que ha interactuado un usuario. Consideraremos que si el
# usuario ha interactuado con la canción, entonces la canción es relevante para el usuario.
# top_n_verdadero_por_usuario = defaultdict(list)
# for i in range(len(sessions)):
#     fila = sessions.iloc[i, :]
#     top_n_verdadero_por_usuario[fila['session_id']].append(fila['track_id'])



In [17]:
user_id = '0_0a9956b4-371b-4085-94d8-662c36df2c7a'
session_tracks = sessions[sessions.user_id == user_id]
tracks_played = session_tracks.merge(track_features, left_on="track_id", right_on="track_id")
tracks_played

,user_id,track_id,not_skipped,Unnamed: 0,duration,year,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0_0a9956b4-371b-4085-94d8-662c36df2c7a,t_529071b7-83da-42bb-819d-0e77c147b5c1,True,29428,0.180673,0.977250,0.003157,0.482673,0.786328,8.857369e-05,7,0.446400,0.095583,0,0.031854,0.579353,4,0.792862
1,0_0a9956b4-371b-4085-94d8-662c36df2c7a,t_529071b7-83da-42bb-819d-0e77c147b5c1,False,29428,0.180673,0.977250,0.003157,0.482673,0.786328,8.857369e-05,7,0.446400,0.095583,0,0.031854,0.579353,4,0.792862
2,0_0a9956b4-371b-4085-94d8-662c36df2c7a,t_df4f8fd6-9e42-43de-81ac-9727b94d9d4d,False,20681,0.136931,0.997527,0.050102,0.839749,0.535920,2.480222e-04,4,0.729726,0.127533,1,0.157888,0.342877,4,0.560035
3,0_0a9956b4-371b-4085-94d8-662c36df2c7a,t_df4f8fd6-9e42-43de-81ac-9727b94d9d4d,False,20681,0.136931,0.997527,0.050102,0.839749,0.535920,2.480222e-04,4,0.729726,0.127533,1,0.157888,0.342877,4,0.560035
4,0_0a9956b4-371b-4085-94d8-662c36df2c7a,t_759c55eb-d225-4ae0-b2b7-c97dd5295f59,False,32623,0.096325,0.997033,0.000866,0.229798,0.934231,3.127147e-04,7,0.109812,0.065983,0,0.103870,0.895418,4,0.401794
5,0_0a9956b4-371b-4085-94d8-662c36df2c7a,t_759c55eb-d225-4ae0-b2b7-c97dd5295f59,False,32623,0.096325,0.997033,0.000866,0.229798,0.934231,3.127147e-04,7,0.109812,0.065983,0,0.103870,0.895418,4,0.401794
6,0_0a9956b4-371b-4085-94d8-662c36df2c7a,t_37e8c595-fbfd-4e29-8afb-f026584822e0,False,10568,0.097529,0.998022,0.618544,0.671784,0.587928,2.409880e-01,1,0.099183,0.159550,0,0.133072,0.772634,4,0.203980
7,0_0a9956b4-371b-4085-94d8-662c36df2c7a,t_37e8c595-fbfd-4e29-8afb-f026584822e0,False,10568,0.097529,0.998022,0.618544,0.671784,0.587928,2.409880e-01,1,0.099183,0.159550,0,0.133072,0.772634,4,0.203980
8,0_0a9956b4-371b-4085-94d8-662c36df2c7a,t_37e8c595-fbfd-4e29-8afb-f026584822e0,False,10568,0.097529,0.998022,0.618544,0.671784,0.587928,2.409880e-01,1,0.099183,0.159550,0,0.133072,0.772634,4,0.203980
9,0_0a9956b4-371b-4085-94d8-662c36df2c7a,t_b3142a9c-a96a-4591-adc0-ba43bbbeb86b,False,50560,0.095397,0.998022,0.011093,0.474737,0.879878,1.896057e-08,11,0.161049,0.059533,0,0.037368,0.767894,4,0.402614


Basado en práctico Content Based (Texto) (https://github.com/PUC-RecSys-Class/RecSysPUC-2022/blob/master/practicos/Content_Based_texto.ipynb)

In [11]:
def find_similar_songs(sessions, track_features, alpha, beta, user_id=None, metric='euclidean', topk=5):
    
    session_tracks = sessions[sessions.user_id == user_id]
    tracks_played = session_tracks.merge(track_features, left_on="track_id", right_on="track_id")
    tracks_not_played = track_features[~track_features.track_id.isin(tracks_played)]
    initial_track = np.array(tracks_played.iloc[0].drop(labels=["user_id", "track_id", "not_skipped"]))
    
    embedding = tracks_not_played.drop(labels="track_id", axis=1).values
    not_played_distances = pairwise_distances(initial_track.reshape(1,-1), embedding, metric=metric)
    tracks_not_played["distance"] = not_played_distances[0]
    tracks_not_played.sort_values(by=["distance"])
    
    embedding= tracks_played.drop(labels=["user_id", "track_id", "not_skipped"], axis=1).values
    played_distances = pairwise_distances(initial_track.reshape(1,-1), embedding, metric=metric)
    tracks_played["distance"] = played_distances[0]
    tracks_played.sort_values(by=["distance"])
    
    rec = []
    
    for i in range(topk):
        # No escuchados
        if rd.random() < alpha:
            loc = int(beta*len(tracks_not_played)//1)
            track = tracks_not_played.iloc[loc]
            while track.track_id in rec:
                loc += 1
                track = tracks_not_played.iloc[loc]
            rec.append(track.track_id)
        # Ya escuchados
        else:
            loc = int(beta*len(tracks_played)//1)
            if loc == 0:
                loc += 1
            track = tracks_played.iloc[loc]
            while track.track_id in rec:
                loc += 1
                track = tracks_played.iloc[loc]
            rec.append(track.track_id)
    
    return rec

In [12]:
random_session = rd.choice(sessions.user_id.unique())
random_session

'0_0a9956b4-371b-4085-94d8-662c36df2c7a'

In [13]:
alpha = widgets.FloatSlider(min=0, max=1, step=0.01, description="alpha")
beta = widgets.FloatSlider(min=0, max=1, step=0.01, description="beta")
display(alpha)
display(beta)

FloatSlider(value=0.0, description='alpha', max=1.0, step=0.01)

FloatSlider(value=0.0, description='beta', max=1.0, step=0.01)

In [14]:
rec = find_similar_songs(sessions, track_features, alpha.value, beta.value, user_id=random_session, \
                         metric='euclidean', topk=5)
rec

['t_529071b7-83da-42bb-819d-0e77c147b5c1',
 't_df4f8fd6-9e42-43de-81ac-9727b94d9d4d',
 't_759c55eb-d225-4ae0-b2b7-c97dd5295f59',
 't_37e8c595-fbfd-4e29-8afb-f026584822e0',
 't_b3142a9c-a96a-4591-adc0-ba43bbbeb86b']

In [15]:
rec = find_similar_songs(sessions, track_features, alpha.value, beta.value, user_id=random_session, \
                         metric='cosine', topk=5)
rec

['t_71be7431-1d03-4df4-a25a-5a5eacc28af4',
 't_8794567a-ea01-4bc7-bc57-039ffead55df',
 't_3f22717f-939c-46db-a5d8-adedaa02b9a7',
 't_00f1b811-1738-4985-a03e-dee72f36adad',
 't_f3a8c230-c5b7-473c-9ec8-7ca0002e6b52']